In [ ]:
import graphlab as gl
from IPython.display import Image
## Set a variable for the path to your (shared) data and/or model directory
path_to_dir = '../data/'

# Building an Image Simiarlity Service
----------

Who doesn't love dresses? Dresses can be incredibly diverse in terms of look, fit, feel, material, trendiness, and quality. They help us feel good, look good, and express ourselves. Needless to say, buying and selling dresses is a big deal. Depending on who you are, what you need, what you want, and what you can afford, this can be really hard. Sometimes it's just hard because you like everything and need to make a decision.

In the keynote demo, we saw an end-to-end application that finds similar items based on text and image metadata. We're going to focus on one of the core parts of building such an application, Image Similarity, that uses transfer learning and nearest neighbours to extract and compare features on images and determine how similar they are.


This iPython notebook consists of the following parts:
1. Loading the Data - loading existing data into an SFrame.
2. Extracing the features - loading an existing pre-trained model into a model object and using it to extract features.
3. Calculating the Distance - Creating and training a k-nearest neighbors classification model on the extracted features.
4. Finding Similar Items - Using the k-nearest neighbors model to help us find similar items.
5. Saving our new model for use in deploying a predictive service! (This is covered in a separate notebook)

<img src='images/workflow1.png'></img>


### Prerequisites:

Make sure you've followed the handout and have Graphlab-Create installed, along with the data/model needed for this tutorial. Please let a volunteer know if you have any trouble.

# Step 1: Load the Data
----------
In this first part, we will introduce two things:
1. A rich product dataset including images and product metadata (data from the keynote demo!).
2. A deep neural network model trained on over 1.2 million images from the ImageNet 2012 competition.


### Load images from product dataset
This dataset includes product metadata (descriptions, category information, price, brand, and image features), links, and some features engineered from other columns (bag of words, TFIDF, etc.). We're loading this data into an SFrame.

In [ ]:
## Creating the SFrame with our path to the directory where it is saved.
image_sf = gl.SFrame(path_to_dir + 
                     'sf_processed.sframe'
                    )

image_sf
#image_sf.show()  #Explore the data using Canvas visual explorer

###Load an existing deep learning model
Here we load the model that will be used to extract visual features extracted via a trained deep neural network. This is an AlexNet architecture that has been trained on the ImageNet data set for 45 iterations.

<img src='images/AlexNet.png'></img>

In [ ]:
pretrained_model = gl.load_model(path_to_dir + 
                                 'imagenet_model')

Of course, when you implement your own transfer-learning solutions, you will have to train your model on a dataset that fits your problem space, whether that's ImageNet data from a given year or something else. You'll benefit most from the generality of the features you extract from your trained model if the data is comprehensive. You can learn how to build the ImageNet model we used for this application on the Dato website <a href='https://dato.com/learn/gallery/notebooks/build_imagenet_deeplearning.html'>here</a>.

In [ ]:
## Let's take a look at the network topology of hte pretrained model
pretrained_model['network']

#Step 2: Extract Features


<img src='images/workflow2.png'></img>


We will be using deep visual features to match the product images to each other. In order to do that, we need to load in our pre-trained ImageNet neural network model to be used as a feature extractor, and extract features from the images in the dataset. 

<img src='images/feature_extraction.png'></img>

To learn more about feature extraction, read this [blog post](http://blog.dato.com/deep-learning-blog-post).

As an example extract the features of the first image in the dataset. The <code>extract_features</code> takes the output from the second last layer of the pretrailed_model, before the classification layer discards the rich features the network has learned.

We also observe the size of the features extracted, compared with the original image:

- 196608: bytes values per 256x256pixel image with an RGB component.
- 4096: size of the penultimate fully-connected layer and hence the # of features 4096.

In [ ]:
image_sf['image'][:10].show()

In [ ]:
# Here are the features of the first image
extracted = pretrained_model.extract_features(image_sf[['image']][:1])
# NB: image_sf is an SFrame, image_sf['image'] is an SArray, and image_sf[['image']] is an SFrame
extracted

We now extract the features for all the images in our dataset using `extract_features`. This does the following:

1. For each image in our dataset, we progagate it through our pre-trained neural network.
2. At each layer of the neural network, some or all neurons are excited -- to various degrees -- by our image.
3. We can represent the excitement of every neuron in a given layer as a vector of all of their excitements.
4. There's an additional parameter `layer_id`, that allows you to choose any fully-connected layer to extract features from. The default is `layer_id=None`, which returns features from the penultimate layer of the pre-trained DNN of your choosing.

In [ ]:
extracted_features = pretrained_model.extract_features(image_sf[['image']]) #Caution! this may take a while
image_sf['features'] = extracted_features ## adding the extracted_features to our SFrame

# Step 3: Calculating the Distance
<img src='images/workflow3.png'></img>

This is the last step in building the similar items recommdendation model. Using the features we extracted above, we are going to create a *k*-Nearest Neighbors model that measures the distance between all our features enables end users to find products whose images match most closely.

<img src='images/nearest_neighbors.png'></img>

In [ ]:
nn_model = gl.nearest_neighbors.create(image_sf, 
                                       features=['features'])  # We're using the pre-extracted features

#Find Similar Items
<img src='images/workflow4.png'></img>

### Query 1: Blue, da ba dee da ba die

In [ ]:
blue = image_sf[194:195]
blue['image'].show()

In [ ]:
integer = 42 ##number of nearest neighbors you want to query, can be in the range(1, len(image_sf))

similar_to_blue = nn_model.query(blue, 
                                 k=integer, 
                                 verbose=True)

similar_to_blue

In [ ]:
## To get the images, we need to join the reference label in our kNN model to our main SFrame
blue_images = image_sf.join(similar_to_blue, 
                            on={'_id':'reference_label'}
                           ).sort('distance')

## Let's show just the first 10 nearest neighbors
blue_images['image'][0:10].show() ## notice how we're taking a length 10 slice of the query we did above, of size 42.

### Query 2: Similarity with more unique images

In [ ]:
interesting = image_sf[2:3]
interesting['image'].show()

In [ ]:
similar_to_interesting = nn_model.query(blue, k=10, verbose=True)
similar_to_interesting

In [ ]:
## To get the images, we need to join the reference label in our kNN model to our main SFrame
interesting_images = image_sf.join(similar_to_interesting, 
                                   on={'_id':'reference_label'}
                                  ).sort('distance')

## Let's show just the first 10 nearest neighbors
interesting_images['image'][0:10].show() ## notice how we're taking a length 10 slice of the query we did above, of size 42.

## Save the model for deploying a predictive service

Now we are going to save the nearest neighbors model we've created and trained, in order to create, run, and deploy a predictive service that you can interact with via a RESTful API.

In [ ]:
nn_model.save('nearest_dress_model')

## Exercises

Now that we've shown how to take an image, query the kNN model for some number of that image's nearest neigbors, and then use our original dataset to find the images for those neighbors, we want you to try to find something new.

### Exercise 1: Least similar to Blue

Find the dress that is LEAST similar to the dress we found above, which, for the record, is:

```python
blue = image_sf[194:195]
blue['image'].show()
```

### Exercise 2: What is the most average dress?

Find the dress that is the most AVERAGE across all of the dresses, in terms of its extracted visual features.

Remember that we stored these visual features in our SFrame `image_sf` in the `extracted_features` column.

### Exercise 3: What is the most unique dress?

Find the dress that is the most unique across all of the dresses, in terms of its extracted visual features.

Remember that we stored these visual features in our SFrame `image_sf` in the `extracted_features` column.

**HINT:** which dress is, in aggregate, farthest from every other dress?

**NOTE:** In Exercise 2, we found the most average dress. You may be thinking, "wouldn't the most unique dress be the dress whose features are furthest from the average?" For some datasets, this may be true, but in general, it is not.

## Exercise Solutions

### Solution to Exercise 1:

In [ ]:
blue = image_sf[194:195] ## Here's our image
blue_neighbors = nn_model.query(blue, 
                                k=len(image_sf), 
                                verbose=True) ## Query the kNN model for all neighbors

blue_neighbor_images = image_sf.join(blue_neighbors, 
                                     on={'_id':'reference_label'}
                                    ).sort('distance') ## Do the join

blue_neighbor_images['image'][-1:].show() ## The least similar image is the one whose distance is greatest

### Solution to Exercise 2:

In [ ]:
mean_features = image_sf['features'].mean() # get the mean vector

# Add a column, with all values as mean vector
image_sf['mean_features'] = image_sf.apply(lambda x: mean_features)

# Center all features on the mean features vector
image_sf['centered_features'] = image_sf['features'] - image_sf['mean_features']

# Find magnitude of all centered vectors (i.e., Euclidean distance from the average feature vector)
from numpy import linalg ## good linear algebra library
image_sf['from_center'] = image_sf['centered_features'].apply(lambda x: linalg.norm(x)) ## norm is Euclidean

# Sort the SFrame by smallest to largest
# The centered_features vector with the smallest magnitude is  visually the most "average" dress in the dataset.
image_sf.sort('from_center')[0:1].show()

### Solution to Exercise 3:

In [ ]:
## Create SFrame of all nearest neighbors for all dresses
## This could take a few minutes (consider using Dato Distributed)
all_nn = nn_model.query(image_sf, 
                        k=len(image_sf), 
                        verbose=True)

## Now, for each dress, take the sum of all distances from it to all other dresses
total_distances = all_nn.groupby(key_columns='query_label', 
                                 operations = {'sum' : gl.aggregate.SUM('distance')})

## Find the label for the most unique dress, i.e., the dress with the highest sum of inter-dress distances
unique = total_distances.sort('sum')[-1]['query_label']

## Let's see what we have
image_sf[unique:unique+1].show()

In [ ]:
image_sf[unique:unique+1].show()

# Bibliography
----------

Olga Russakovsky*, Jia Deng*, Hao Su, Jonathan Krause, Sanjeev Satheesh, Sean Ma, Zhiheng Huang, Andrej Karpathy, Aditya Khosla, Michael Bernstein, Alexander C. Berg and Li Fei-Fei. (* = equal contribution) ImageNet Large Scale Visual Recognition Challenge. arXiv:1409.0575, 2014.

Donahue, J., Jia, Y., Vinyals, O., Homan, J., Zhang, N., Tzeng, E., and Darrell, T. DeCAF: A deep convolutional activation feature for generic visual recognition. In JMLR, 2014.

Krizhevsky, Alex, Ilya Sutskever, and Geoffrey E. Hinton. "Imagenet classification with deep convolutional neural networks." In Advances in neural information processing systems, pp. 1097-1105. 2012.

Yang, J., L., Y., Tian, Y., Duan, L., and Gao, W. Group-sensitive multiple kernel learning for object categorization. In ICCV, 2009.

Szegedy, Christian, Wojciech Zaremba, Ilya Sutskever, Joan Bruna, Dumitru Erhan, Ian Goodfellow, and Rob Fergus. "Intriguing properties of neural networks." arXiv preprint arXiv:1312.6199 (2013).

Nguyen, Anh, Jason Yosinski, and Jeff Clune. "Deep neural networks are easily fooled: High confidence predictions for unrecognizable images." arXiv preprint arXiv:1412.1897 (2014).

Goodfellow, Ian J., Jonathon Shlens, and Christian Szegedy. "Explaining and harnessing adversarial examples." arXiv preprint arXiv:1412.6572 (2014).